# No ambiente do colab precisaremos do apache spark, uma versão do java e iremos usar o findspark

inicio da instalação do spark



In [2]:
# você utilizar mesmos procedimentos no linux
# no redhat linux rpm
# no debian, ubuntu você pode usar nativamente o apt-get
# sudo apt-get

# vamos usar um comando que é para o linux chamado apt-get e para instalar no colab você coloca !
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
# wget que irei usar para baixar o spark
# url do apache spark
!wget -q https://downloads.apache.org/spark/spark-2.4.7/spark-2.4.7-bin-hadoop2.7.tgz

In [4]:
# descompactar o arquivo spark-2.4.7-bin-hadoop2.7.tgz
!tar xf spark-2.4.7-bin-hadoop2.7.tgz

In [5]:
!pip install -q findspark

In [7]:
import os

os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['SPARK_HOME'] = "/content/spark-2.4.7-bin-hadoop2.7"

finalizada a instalação

In [8]:
import findspark
findspark.init()

In [9]:
# importar a classe SparkSession
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [10]:
spark

In [15]:
# importando classes
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

In [24]:
# ler o dataset
# /content/drive/My Drive/BostonHousing.csv

# inferSchema=True para inferir os tipos de dados
# header=True para retornar o cabeçalho
dados = spark.read.csv('/content/drive/My Drive/BostonHousing.csv', header=True, inferSchema=True)

In [25]:
dados.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5605|  5|311|   15.2| 395.6|12.43|22.9|
|0.14455|12.5| 7.87|   0|0.524|6.172| 96.1|5.9505|  5|311|   15.2| 396.9|19.15|27.1|
|0.21124|12.5| 7.87|   0|0.524|5.631|100.0|6.0821|  5|311|   15.2

In [27]:
dados.printSchema()

root
 |-- crim: double (nullable = true)
 |-- zn: double (nullable = true)
 |-- indus: double (nullable = true)
 |-- chas: integer (nullable = true)
 |-- nox: double (nullable = true)
 |-- rm: double (nullable = true)
 |-- age: double (nullable = true)
 |-- dis: double (nullable = true)
 |-- rad: integer (nullable = true)
 |-- tax: integer (nullable = true)
 |-- ptratio: double (nullable = true)
 |-- b: double (nullable = true)
 |-- lstat: double (nullable = true)
 |-- medv: double (nullable = true)



In [29]:
entradas = dados.columns

In [31]:
# precisamos colocar as features as entradas em formato de Vetor
vetor_entradas = VectorAssembler(inputCols=dados.columns, outputCol='Entradas')

In [35]:
dados_transformados = vetor_entradas.transform(dados)

In [36]:
dados_transformados.show()

+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|   crim|  zn|indus|chas|  nox|   rm|  age|   dis|rad|tax|ptratio|     b|lstat|medv|            Entradas|
+-------+----+-----+----+-----+-----+-----+------+---+---+-------+------+-----+----+--------------------+
|0.00632|18.0| 2.31|   0|0.538|6.575| 65.2|  4.09|  1|296|   15.3| 396.9| 4.98|24.0|[0.00632,18.0,2.3...|
|0.02731| 0.0| 7.07|   0|0.469|6.421| 78.9|4.9671|  2|242|   17.8| 396.9| 9.14|21.6|[0.02731,0.0,7.07...|
|0.02729| 0.0| 7.07|   0|0.469|7.185| 61.1|4.9671|  2|242|   17.8|392.83| 4.03|34.7|[0.02729,0.0,7.07...|
|0.03237| 0.0| 2.18|   0|0.458|6.998| 45.8|6.0622|  3|222|   18.7|394.63| 2.94|33.4|[0.03237,0.0,2.18...|
|0.06905| 0.0| 2.18|   0|0.458|7.147| 54.2|6.0622|  3|222|   18.7| 396.9| 5.33|36.2|[0.06905,0.0,2.18...|
|0.02985| 0.0| 2.18|   0|0.458| 6.43| 58.7|6.0622|  3|222|   18.7|394.12| 5.21|28.7|[0.02985,0.0,2.18...|
|0.08829|12.5| 7.87|   0|0.524|6.012| 66.6|5.5

In [37]:
# X Entradas são as features no formato de vetor utiliando VectorAssembler
# Y label alvo medv
dados_final = dados_transformados.select('Entradas', 'medv')

In [39]:
dados_final.show()

+--------------------+----+
|            Entradas|medv|
+--------------------+----+
|[0.00632,18.0,2.3...|24.0|
|[0.02731,0.0,7.07...|21.6|
|[0.02729,0.0,7.07...|34.7|
|[0.03237,0.0,2.18...|33.4|
|[0.06905,0.0,2.18...|36.2|
|[0.02985,0.0,2.18...|28.7|
|[0.08829,12.5,7.8...|22.9|
|[0.14455,12.5,7.8...|27.1|
|[0.21124,12.5,7.8...|16.5|
|[0.17004,12.5,7.8...|18.9|
|[0.22489,12.5,7.8...|15.0|
|[0.11747,12.5,7.8...|18.9|
|[0.09378,12.5,7.8...|21.7|
|[0.62976,0.0,8.14...|20.4|
|[0.63796,0.0,8.14...|18.2|
|[0.62739,0.0,8.14...|19.9|
|[1.05393,0.0,8.14...|23.1|
|[0.7842,0.0,8.14,...|17.5|
|[0.80271,0.0,8.14...|20.2|
|[0.7258,0.0,8.14,...|18.2|
+--------------------+----+
only showing top 20 rows



# Divisão entre dados de treino e teste

In [40]:
# 70%(0.7) para treino e 30%(0.3) para teste 
dados_treino, dados_teste = dados_final.randomSplit([0.7, 0.3])

# Criar Modelo de Regressão Linear


In [42]:
# modelo_regressao_linear é uma instância LinearRegression
modelo_regressao_linear = LinearRegression(featuresCol="Entradas", labelCol="medv")

# Treinamento do modelo

In [47]:
modelo_regressao_linear_treinado = modelo_regressao_linear.fit(dados_treino)

In [48]:
modelo_regressao_linear_treinado.coefficients

DenseVector([0.0, -0.0, 0.0, -0.0, 0.0, -0.0, -0.0, 0.0, -0.0, 0.0, 0.0, -0.0, 0.0, 1.0])

In [49]:
modelo_regressao_linear_treinado.intercept

-9.498681757250184e-14

# Avaliar o modelo


In [45]:
previsao = modelo_regressao_linear_treinado.evaluate(dados_teste)

In [50]:
previsao.predictions.show()

+--------------------+----+------------------+
|            Entradas|medv|        prediction|
+--------------------+----+------------------+
|[0.0136,75.0,4.0,...|18.9|18.900000000000006|
|[0.01439,60.0,2.9...|29.1|29.099999999999994|
|[0.01778,95.0,1.4...|32.9|32.900000000000006|
|[0.01965,80.0,1.7...|20.1|20.099999999999998|
|[0.02498,0.0,1.89...|16.5|16.499999999999982|
|[0.02543,55.0,3.7...|23.9|23.899999999999984|
|[0.02729,0.0,7.07...|34.7| 34.70000000000002|
|[0.02763,75.0,2.9...|30.8|30.799999999999997|
|[0.02899,40.0,1.2...|26.6|26.600000000000012|
|[0.03041,0.0,5.19...|18.5|18.499999999999996|
|[0.03237,0.0,2.18...|33.4| 33.40000000000002|
|[0.03359,75.0,2.9...|34.9|              34.9|
|[0.03466,35.0,6.0...|19.4|19.399999999999988|
|[0.0351,95.0,2.68...|48.5| 48.50000000000002|
|[0.03537,34.0,6.0...|22.0| 21.99999999999998|
|[0.03584,80.0,3.3...|23.5| 23.49999999999998|
|[0.03615,80.0,4.9...|27.9|27.899999999999988|
|[0.03705,20.0,3.3...|35.4|              35.4|
|[0.03871,52.

In [51]:
previsao.meanAbsoluteError

9.798429873743049e-15

In [53]:
previsao.meanSquaredError

2.062416155092242e-28

In [54]:
previsao.rootMeanSquaredError

1.4361114702878193e-14